# Hypotheses Testing

### H1 - Service quality drives satisfaction more than operational metrics

What we test is that is average service score significantly different between satisfied and dissatisfied passengers?